In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard

In [2]:
from google.cloud import datastore
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"


client = datastore.Client()
query = client.query(kind="pricedata")
x = []
y = []
for i in query.fetch():
    x.append([i['smolprice'], i['chonkprice'], i['absprice'],i["balance"]])
    y.append([i['coins']])


In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(2, activation='relu',input_shape=(4,)))
# Add an output layer with 10 output units:
model.add(layers.Dense(1))

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [5]:
import numpy as np

In [6]:
data = np.asarray(x)
target = np.asarray(y)
print(data)
print(target)
print(data.shape)
print(target.shape)

[[  50  300  500   38]
 [   5  505  555 1204]]
[[904]
 [360]]
(2, 4)
(2, 1)


In [43]:

def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    if std_data == 0:
        norm_data = (data-mean_data)
    else:
        norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train, xmean_data, xstd_data = Normalize(X_train)
X_test, _, _ = Normalize(X_test, xmean_data, xstd_data)

Y_train, ymean_data, ystd_data = Normalize(y_train)
Y_test,_,_ = Normalize(y_test, ymean_data, ystd_data)

In [44]:
print(X_train)
print(Y_train)

[[-1.32011145 -0.14615729 -0.02876188  1.49503062]]
[[0.]]


In [45]:
print(X_test)
print(Y_test)

[[-1.21445558 -0.6274785  -0.15789683 -1.24263048]]
[[544.]]


In [46]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train, Y_train,
          validation_data=(X_test,Y_test), 
          batch_size=20, 
          epochs=25,
          verbose=1,
         callbacks=[tensorboard_callback])

Epoch 1/25
1/1 [==============================] - 0s 365ms/step - loss: 3.3199 - mae: 1.8221 - val_loss: 294066.3125 - val_mae: 542.2788
Epoch 2/25
1/1 [==============================] - 0s 60ms/step - loss: 3.0419 - mae: 1.7441 - val_loss: 294114.8438 - val_mae: 542.3235
Epoch 3/25
1/1 [==============================] - 0s 64ms/step - loss: 2.7791 - mae: 1.6671 - val_loss: 294162.9062 - val_mae: 542.3679
Epoch 4/25
1/1 [==============================] - 0s 56ms/step - loss: 2.5315 - mae: 1.5911 - val_loss: 294210.3750 - val_mae: 542.4116
Epoch 5/25
1/1 [==============================] - 0s 64ms/step - loss: 2.2988 - mae: 1.5162 - val_loss: 294257.1875 - val_mae: 542.4548
Epoch 6/25
1/1 [==============================] - 0s 73ms/step - loss: 2.0808 - mae: 1.4425 - val_loss: 294303.2812 - val_mae: 542.4973
Epoch 7/25
1/1 [==============================] - 0s 62ms/step - loss: 1.8771 - mae: 1.3701 - val_loss: 294348.6250 - val_mae: 542.5391
Epoch 8/25
1/1 [==============================]

In [47]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 3         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 9137), started 13:48:22 ago. (Use '!kill 9137' to kill it.)

In [49]:
key = client.key("thoughtcoins", 5713227891605504)
entity = client.get(key=key)
balance = entity["balance"]

In [50]:
balance

138

In [29]:
data = []
maxval = 0
stepsize = 50
maxvaldat = [[5, 5, 5]]
for smol in range(5, 600, stepsize):
    for chonk in range(5, 600, stepsize):
        for abso in range(5, 600, stepsize):
            x = np.array([[smol, chonk, abso,balance]])
            dat = Normalize(x, xmean_data, xstd_data)[0]
            pred = model.predict(dat)
            data.append([smol,chonk,abso,pred[0][0] *ystd_data + ymean_data])
            if(pred > maxval):
                maxval = pred
                maxvaldat = [[smol, chonk, abso]]


In [30]:
def keyfunc(e):
    return e[3]

In [31]:
data.sort(reverse=True,key=keyfunc)

In [32]:
data

[[5, 5, 5, 360.0],
 [5, 5, 55, 360.0],
 [5, 5, 105, 360.0],
 [5, 5, 155, 360.0],
 [5, 5, 205, 360.0],
 [5, 5, 255, 360.0],
 [5, 5, 305, 360.0],
 [5, 5, 355, 360.0],
 [5, 5, 405, 360.0],
 [5, 5, 455, 360.0],
 [5, 5, 505, 360.0],
 [5, 5, 555, 360.0],
 [5, 55, 5, 360.0],
 [5, 55, 55, 360.0],
 [5, 55, 105, 360.0],
 [5, 55, 155, 360.0],
 [5, 55, 205, 360.0],
 [5, 55, 255, 360.0],
 [5, 55, 305, 360.0],
 [5, 55, 355, 360.0],
 [5, 55, 405, 360.0],
 [5, 55, 455, 360.0],
 [5, 55, 505, 360.0],
 [5, 55, 555, 360.0],
 [5, 105, 5, 360.0],
 [5, 105, 55, 360.0],
 [5, 105, 105, 360.0],
 [5, 105, 155, 360.0],
 [5, 105, 205, 360.0],
 [5, 105, 255, 360.0],
 [5, 105, 305, 360.0],
 [5, 105, 355, 360.0],
 [5, 105, 405, 360.0],
 [5, 105, 455, 360.0],
 [5, 105, 505, 360.0],
 [5, 105, 555, 360.0],
 [5, 155, 5, 360.0],
 [5, 155, 55, 360.0],
 [5, 155, 105, 360.0],
 [5, 155, 155, 360.0],
 [5, 155, 205, 360.0],
 [5, 155, 255, 360.0],
 [5, 155, 305, 360.0],
 [5, 155, 355, 360.0],
 [5, 155, 405, 360.0],
 [5, 155, 455

In [35]:
newprice = random.choice(data)
newprice

[205, 255, 105, 360.0]

In [36]:
client = datastore.Client()
key = client.key('prices', 5631671361601536)
entity = datastore.Entity(key=key)
entity.update({
    "smolprice": newprice[0],
    "chonkprice": newprice[1],
   "absprice": newprice[2]    })
client.put(entity)

## import matplotlib.pyplot as plt

In [111]:
(8-2)/3

2.0

In [112]:
2.0*3+2

8.0